In [ ]:
import os
import numpy as np
import pandas as pd
import torch

from MainTrain import Main, MainOptuna
from modules.sampling import SamplerContextMatrix, SamplerRandomWalk, SamplerFactorization, SamplerAPP

In [ ]:
synthetic = True
if synthetic:
    datasets_names=[]
    for l_a_trgt in [0.5]:
                for f_a_trgt in [0.1,0.5,0.9]:
                    for cl_trgt in [0.01,0.1,0.2,0.3,0.5]:
                        for asp_trgt in [2,3,4,5,6,7]:
                            for a_deg_trgt in [2,5,10,15,20,25,30,35,40]:
                                datasets_names.append((l_a_trgt,f_a_trgt,cl_trgt,asp_trgt,a_deg_trgt))

else:
    datasets_names = ['Cornell','Texas','Wisconsin','Actor','Pubmed','squirrel']

In [ ]:
#loss functions
VERSE_PPR =  {"Name": "VERSE_PPR","C": "PPR","num_negative_samples":[1, 6, 11, 16, 21],"loss var": "Context Matrix","flag_tosave":False,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerContextMatrix,"lmbda": [0.0,1.0]}
VERSE_Adj =  {"Name": "VERSE_Adj","C": "Adj","num_negative_samples":[1, 6, 11, 16, 21],"loss var": "Context Matrix","flag_tosave":False,"Sampler" :SamplerContextMatrix,"lmbda": [0.0,1.0]}

VERSE_SR =  {"Name": "VERSE_SimRank","C": "SR","num_negative_samples":[1, 6, 11, 16, 21],"loss var": "Context Matrix","flag_tosave":False,"Sampler":SamplerContextMatrix,"lmbda": [0.0,1.0]}
DeepWalk = {"Name": "DeepWalk","walk_length":[5, 10, 15, 20],"walks_per_node":[5, 10, 15, 20],"num_negative_samples":[1,6, 11, 16, 21],"context_size" : [5, 10, 15, 20],"p":1,"q":1,"loss var": "Random Walks","flag_tosave":False,"Sampler" : SamplerRandomWalk } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk_length":[5, 10, 15, 20],"walks_per_node":[5, 10, 15, 20],"num_negative_samples":[1,6, 11, 16, 21],"context_size" : [5, 10, 15, 20],"p": [0.25, 0.50, 1, 2, 4] ,"q":[0.25, 0.50, 1, 2, 4], "loss var": "Random Walks","flag_tosave":False,"Sampler": SamplerRandomWalk}#то же самое
APP ={"Name": "APP","C": "PPR","num_negative_samples":[1, 6, 11, 16, 21],"loss var": "Context Matrix","flag_tosave":True,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerAPP}
HOPE_Katz = {"Name": "HOPE_Katz","C":"Katz","loss var": "Factorization","flag_tosave":True,"betta": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} #проверить

HOPE_RPR = {"Name": "HOPE_RPR","C":"RPR","loss var": "Factorization","flag_tosave":True,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} #проверить
HOPE_CN = {"Name": "HOPE_CommonNeighbors", "C":"CN","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]}
HOPE_AA = {"Name": "HOPE_AdamicAdar","C":"AA","loss var": "Factorization","flag_tosave":True,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]}

LapEigen = {"Name": "LaplacianEigenMaps", "C":"Adj","loss var": "Laplacian EigenMaps","flag_tosave":True,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]}
LINE = {"Name": "LINE","C": "Adj","num_negative_samples":[1, 6, 11, 16, 21],"loss var": "Context Matrix","flag_tosave":False,"Sampler" :SamplerContextMatrix,"lmbda": [0.0,1.0]}
GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]}

Force2Vec = {"Name": "Force2Vec","C": "Adj","num_negative_samples":[1, 6, 11, 16, 21],"loss var": "Force2Vec","flag_tosave":False,"Sampler" :SamplerContextMatrix,"lmbda": [0.0,1.0]}


In [ ]:
loss = HOPE_CN
loss_name = 'HOPE_CN'
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
analysis = pd.read_csv('../results/whole_data_results.csv')
analysis = analysis.drop(columns=['Unnamed: 0'])
device='cpu'

for (l,f,cl,asp,ad) in datasets_names:
    for conv in ['SAGE','GCN','GAT']:
        name = "".join(list(map(lambda x: str(x), [l, f, cl, asp, ad])))
        if os.path.exists('../data_benchmark/graph_' + str(name) + '_attr.npy'):

            if len(analysis[ (analysis['loss'] == loss_name) & (analysis['conv'] == conv) & (analysis['dataset'] == name)] ) == 0:
                print(l,f,cl,asp,ad,conv)
                MO = MainOptuna(name=name,conv=conv, device=device, loss_function=loss,mode= 'unsupervised')
                study = MO.run(number_of_trials=5)
                best_values = study.best_trial.params

                columns = ["Loss name", "Conv", "l", "f","cl", "asp", "ad", "loss value"] + list(study.trials[0].params.keys())
                df = pd.DataFrame(columns=columns)
                for trial in study.trials:
                    row = [loss_name,conv,l,f,cl,asp,ad,trial.values[0]] + list(trial.params.values())
                    to_append = pd.Series(row,df.columns)
                    df = df.append(to_append,ignore_index=True)
                df.to_csv(str(loss_name)+'_'+str(conv)+'_'+str(name)+'.csv')

                loss_trgt=dict()
                for par in loss:
                    loss_trgt[par]=loss[par]

                loss_trgt["lmbda"] = best_values['lmbda']

                M = Main(name=name,conv=conv, device=device, loss_function=loss_trgt,mode= 'unsupervised')
                accuracy = M.run(best_values)

                to_append=pd.Series([loss_name, conv, f,cl,asp,ad,name , accuracy, best_values],index = analysis.columns)
                analysis = analysis.append(to_append,ignore_index=True)
                print(analysis)